In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('../../raw/LiChuan/trainSaleDate.csv')
test = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')
trainFeature = pd.read_csv('../../raw/LiChuan/train_feature.csv')

In [3]:
test['year'] = test['predict_date'].apply(lambda x: int(str(x)[0:4]))
test['month'] = test['predict_date'].apply(lambda x: int(str(x)[4:6]))
# test = test.drop(['predict_date'], axis=1)
test.head()

,predict_date,class_id,predict_quantity,year,month
0,201711,103507,NaN,2017,11
1,201711,124140,NaN,2017,11
2,201711,125403,NaN,2017,11
3,201711,136916,NaN,2017,11
4,201711,169673,NaN,2017,11


In [4]:
sellmonth = train[['class_id', 'how_many_month_has_sell', 'sale_quantity']][(train['year']==2017)&(train['month']==10)]
sellmonth.dropna(inplace=True) # inplace=True 指在原 dataframe 上修改数据, 而非返回新的
sellmonth['how_many_month_has_sell'] = sellmonth['how_many_month_has_sell']+1
sale_quantity_total_last_month = sellmonth['sale_quantity'].sum()
sellmonth.rename(columns={'sale_quantity': 'sale_quantity_this_last_month'}, inplace=True)

sellYear = train[['class_id', 'sale_quantity']][((train['year']==2016)&(train['month']>=11))|((train['year']==2017)&(train['month']<11))]
sellYear.dropna(inplace=True)
sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)

test = pd.merge(test, trainFeature, on='class_id', how='left')
test = pd.merge(test, sellmonth, on='class_id', how='left')
test = pd.merge(test, sellYear, on='class_id', how='left')

test.loc[:, 'sale_quantity_total_last_month'] = sale_quantity_total_last_month
test.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
test.loc[:, 'this_month_sale_class_count'] = 140

test.sort_values(['class_id'])
test.fillna(0.0, inplace=True)

test = test.drop(['predict_quantity'], axis=1)
test.head()

,predict_date,class_id,year,month,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,...,rated_passenger_6-8,rated_passenger_7,rated_passenger_7-8,rated_passenger_9,how_many_month_has_sell,sale_quantity_this_last_month,sale_quantity_this_last_year,sale_quantity_total_last_month,sale_quantity_total_last_year,this_month_sale_class_count
0,201711,103507,2017,11,0,0,0,0,0,0,...,0,0,0,0,33.0,180.0,5783.0,50076.0,644768.0,140
1,201711,124140,2017,11,0,0,0,0,0,0,...,0,0,0,0,43.0,263.0,3322.0,50076.0,644768.0,140
2,201711,125403,2017,11,0,0,0,0,0,0,...,0,1,0,0,71.0,156.0,3138.0,50076.0,644768.0,140
3,201711,136916,2017,11,0,0,0,0,0,0,...,0,0,0,0,71.0,162.0,2217.0,50076.0,644768.0,140
4,201711,169673,2017,11,0,0,0,0,0,0,...,0,1,0,0,6.0,152.0,581.0,50076.0,644768.0,140


In [5]:
columns = train.drop(['sale_quantity'], axis=1).columns.tolist()

test['sale_date'] = test['predict_date']

test.drop(['predict_date'], axis=1)

,class_id,year,month,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,...,rated_passenger_7,rated_passenger_7-8,rated_passenger_9,how_many_month_has_sell,sale_quantity_this_last_month,sale_quantity_this_last_year,sale_quantity_total_last_month,sale_quantity_total_last_year,this_month_sale_class_count,sale_date
0,103507,2017,11,0,0,0,0,0,0,0,...,0,0,0,33.0,180.0,5783.0,50076.0,644768.0,140,201711
1,124140,2017,11,0,0,0,0,0,0,0,...,0,0,0,43.0,263.0,3322.0,50076.0,644768.0,140,201711
2,125403,2017,11,0,0,0,0,0,0,0,...,1,0,0,71.0,156.0,3138.0,50076.0,644768.0,140,201711
3,136916,2017,11,0,0,0,0,0,0,1,...,0,0,0,71.0,162.0,2217.0,50076.0,644768.0,140,201711
4,169673,2017,11,0,0,0,0,0,0,0,...,1,0,0,6.0,152.0,581.0,50076.0,644768.0,140,201711
5,175962,2017,11,0,0,0,0,0,0,0,...,0,0,0,60.0,237.0,3009.0,50076.0,644768.0,140,201711
6,178529,2017,11,0,0,0,0,0,1,0,...,1,0,0,71.0,148.0,2867.0,50076.0,644768.0,140,201711
7,186250,2017,11,0,0,0,0,0,0,0,...,0,0,0,22.0,83.0,1753.0,50076.0,644768.0,140,201711
8,194201,2017,11,0,0,0,0,0,0,0,...,0,0,0,3.0,378.0,617.0,50076.0,644768.0,140,201711
9,194450,2017,11,0,0,0,0,0,0,0,...,0,0,0,27.0,271.0,3063.0,50076.0,644768.0,140,201711


In [6]:
test = test[columns]
test.to_csv('../../raw/LiChuan/test_feature.csv', index=False)